In [5]:
// 先引入要使用的库
%use kandy
%use lib-ext
%use dataframe

先考虑一维的情况.


In [6]:
import kotlin.random.Random

val listSize = 128
val line = Array(listSize + 1) { 0.0 }

line[0] =   randomHeight(listSize.toDouble())
line[listSize] =  randomHeight(listSize.toDouble())

// 工具函数
fun randomHeight(len: Double): Double {
    val value = Random.nextDouble(-len / 2, len / 2)
    return value
}


中间点值没有随机值就是直线。

In [7]:

fun mid(start: Int, end: Int) {
    if (end - start == 1) return;

    val midIndex = (start + end) / 2
    line[midIndex] = (line[start] + line[end]).toDouble() / 2
    mid(start, midIndex)
    mid(midIndex, end)
}

mid(0, listSize)

plot(mapOf(
    "index" to (0..listSize).toList(),
    "line" to line.toList(),
)) {
    points {
        x("index")
        y("line")
    }
    
    line {
        x("index")
        y("line")
    }
}

生成一个跟宽度相关随机序列，为了固定后面的计算结果，方便对比


```
[0, ..., (-len/4~len/4), ... (-len/2~len/2), ...  (-len/4~len/4), ..., 0]

```

这个算法的核心点在于：中间点的值 = (起点 + 终点) / 2 + （-区间长度/2 ~ 区间长度/2）之间的随机值

假设有列表：list = (0, 0, 0, 0, 0)

第一次迭代，list\[2\]生成随机值的区间是 (-2 ~ 2) 假设生成随机值 1.8。

结果是：list = (0, 0, 1.8, 0, 0)

第二次迭代：list\[1\] list\[3\]生成随机值的区间是 (-1 ~ 1)，加上生成的随机值是 0.6、-0.4

```
list[1] = (list[0] + list[2]) / 2 + 0.6
        = (0 + 1.8) / 2 + 0.6
        = 1.5

list[3] = (list[2] + list[4]) / 2 + 0.6
        = (0 + 1.8) / 2 + -0.4
        = 0.5
```

最终结果是：

list = (0, 1.5, 1.8, -0.4, 0)


”区间长度“ 限制了中间的摆动服务，避免生成的曲线过于畸形。

In [24]:
val sampleSize = 33
val sampleResults = listOf(0.9, 0.7, 0.5, 0.3, 0.1, -0.1, -0.3, -0.5, -0.7, -0.9).map {
    val sample = Array(sampleSize) { 0.0 }.toMutableList()

    fun mid(len: Int, start: Int, end: Int) {
        if (end - start == 1) return;
        val midIndex = (start + end) / 2
        sample[midIndex] =  len / 2 * it
        mid(len / 2, start, midIndex)
        mid(len / 2, midIndex, end)
    }
    mid(sample.size - 1, 0, sample.size - 1)
    
   "random $it" to sample.toList()
}.toMap().toMutableMap()
 val keys = sampleResults.keys.toList()
 
sampleResults["index"] = (1..sampleSize).toList().map { it.toDouble() }
 
plot(sampleResults) {
    keys.forEach{ key ->
        points {
            x("index")
            y(key)
        }
        line {
            x("index")
            y(key)
        }
    }
}

中间点 = (起点 + 终点)/2 使得在迭代过程中区间内点都会受到起止点的影响。

生成曲线下面区域里任意一条实现

In [38]:
val sampleSize = 33
val sampleResults = listOf(1.0,
                           // 0.9, 0.7, 0.5, 0.3, 0.1, -0.1, -0.3, -0.5, -0.7, -0.9,
                           -1.0).map {
    val sample = Array(sampleSize) { 0.0 }.toMutableList()

    fun mid(len: Int, start: Int, end: Int) {
        if (end - start == 1) return;
        val midIndex = (start + end) / 2
        sample[midIndex] =  (sample[start] + sample[end]).toDouble() / 2 + len / 2 * it
        mid(len / 2, start, midIndex)
        mid(len / 2, midIndex, end)
    }
    mid(sample.size - 1, 0, sample.size - 1)
    
   "random $it" to sample.toList()
}.toMap().toMutableMap()

// 生成随机曲线
repeat(5) {i ->
    val sample = Array(sampleSize) { 0.0 }.toMutableList()

    fun mid(len: Int, start: Int, end: Int) {
        if (end - start == 1) return;
        val midIndex = (start + end) / 2
        sample[midIndex] =  (sample[start] + sample[end]).toDouble() / 2 + Random.nextDouble(-len.toDouble() / 2, len.toDouble() / 2)
        mid(len / 2, start, midIndex)
        mid(len / 2, midIndex, end)
    }
    mid(sample.size - 1, 0, sample.size - 1)

    sampleResults["random $i"] = sample.toList()
}

 val keys = sampleResults.keys.toList()
 
sampleResults["index"] = (1..sampleSize).toList().map { it.toDouble() }

plot(sampleResults) {
    keys.forEach{ key ->
        points {
            x("index")
            y(key)
        }
        
        line {
            x("index")
            y(key)
        }
    }
}

In [31]:
val sampleSize = 33
val sampleResults = listOf(0.9, 0.7, 0.5, 0.3, 0.1, -0.1, -0.3, -0.5, -0.7, -0.9).map {
    val sample = Array(sampleSize) { 0.0 }.toMutableList()

    fun mid(len: Int, start: Int, end: Int) {
        if (end - start == 1) return;
        val midIndex = (start + end) / 2
        sample[midIndex] =  (sample[start] + sample[end]).toDouble() / 2 + len / 2 * it
        mid(len / 2, start, midIndex)
        mid(len / 2, midIndex, end)
    }
    mid(sample.size - 1, 0, sample.size - 1)
    
   "random $it" to sample.toList()
}.toMap().toMutableMap()
 val keys = sampleResults.keys.toList()
 
sampleResults["index"] = (1..sampleSize).toList().map { it.toDouble() }
 
plot(sampleResults) {
    keys.forEach{ key ->
        points {
            x("index")
            y(key)
        }
    }
}

In [32]:
val randomList = Array(listSize + 1) { 0.0 }
val midAverageList = Array(listSize + 1) { 0.0 }

fun fixRandom(len: Int, start: Int, end: Int) {
    if (end - start == 1) return;
    val midIndex = (start + end) / 2
    randomList[midIndex] =  randomHeight(len.toDouble())
    
    midAverageList[midIndex] = (midAverageList[start] + midAverageList[end]).toDouble() / 2 + randomList[midIndex]
    fixRandom(len / 2, start, midIndex)
    fixRandom(len / 2, midIndex, end)
}

fixRandom(listSize, 0, listSize);

plot(mapOf(
    "index" to (0..listSize).toList(),
    "random" to randomList.toList(),
    "midAerage" to midAverageList.toList(),
    "line" to line.toList(),
)) {
    points {
        x("index")
        y("random")
    }
    line {
        x("index")
        y("random")
    }
    points {
        x("index")
        y("midAerage")
        color = Color.GREEN
    }

    line {
        x("index")
        y("midAerage")
        color = Color.GREEN
    }
    points {
        x("index")
        y("line")
        color = Color.YELLOW
    }

    line {
        x("index")
        y("line")
        color = Color.YELLOW
    }
}

中间点的高度是：起点、终点的平均值 + (-len/2 ~ len/)之前的随机值

In [10]:
val line2 = line.toMutableList()
fun mid_v2(len: Int, start: Int, end: Int) {
    if (end - start == 1) return;

    val midIndex = (start + end) / 2
    line2[midIndex] = (line2[start] + line2[end]).toDouble() / 2 + randomList[midIndex]
    mid_v2(len / 2, start, midIndex)
    mid_v2(len / 2, midIndex, end)
}

mid_v2(listSize, 0, listSize)

plot(mapOf(
    "index" to (0..listSize).toList(),
    "line2" to line2.toList(),
    "midAerage" to midAverageList.toList(),
    "random" to randomList.toList(),
)) {
    points {
        x("index")
        y("line2")
    }
    line {
        x("index")
        y("line2")
    }
    
    points {
        x("index")
        y("midAerage")
        color = Color.GREY
    }
    
    line {
        x("index")
        y("midAerage")
        color = Color.GREY
    }
    points {
        x("index")
        y("random")
        color = Color.PEACH
    }
    line {
        x("index")
        y("random")
        color = Color.PEACH
    }
}

算法中有个 roughness 参数，表示光滑程度，越大越平滑、越接近直线

In [11]:

val results = listOf(0.5, 1.0, 1.5, 2.0, 3.0, 5.0).map {
    val roughnessValue = 2.0.pow(-it)

    val line3 = line.toMutableList()
    fun mid_v3(len: Int, start: Int, end: Int) {
        if (end - start == 1) return;
        val midIndex = (start + end) / 2
        line3[midIndex] = (line3[start] + line3[end]).toDouble() / 2 + randomList[midIndex] * roughnessValue
        mid_v3(len / 2, start, midIndex)
        mid_v3(len / 2, midIndex, end)
    }

    mid_v3(listSize, 0, listSize)
    
    plot(mapOf(
        "index" to (0..listSize).toList(),
        "line3" to line3.toList(),
        "line2" to line2.toList(),
    )) {
        points {
            x("index")
            y("line3")
        }
        line {
            x("index")
            y("line3")
        }
        points {
            x("index")
            y("line2")
            color = Color.GREY
        }
        line {
            x("index")
            y("line2")
            color = Color.GREY
        }
        layout.title = "filter $it roughness ${roughnessValue}"

    }
}.toList()

plotGrid(results, nCol = 2)